## Support vector machines - Quadratic programming

In this project we will learn to fit a Support Vector Machine (SVM) model applying quadratic programming methods, this will require to understand the model and know very well the structure of it in order to get a proper optimization model. For purposes of this code the model will be solved using Gurobi with python, you can use other kind of solvers and optimization packages or frameworks that will work in the same way.

### Quadratic programming

First or all we will review what is Quadratic Programming (QP), this in case you are not used to this concept or technique. QP is a process of solving mathematical optimization problems that involves quadratic functions as the objective of the problem, along with linear constraints. This kind of problems are the type of nonlinear programming methods and are a branch of mathematic programs.

The general form of a quadratic programming problem is determined as follows,

\begin{equation}
\text{max} f = \sum_{j=1}^{n} P_j x_j + \sum_{j=1}^{n} \sum_{k=1}^{n} d_{j,k} x_j x_k
\end{equation}

\begin{equation}
\text{subject to:} \sum_{j =1}^{n} a_{i,j}x_j - b_j \leq 0, \ \ x_j \leq 0, \ \ i,j \in \{1,2,...,n \}
\end{equation}

where the quantity $ \sum_{j=1}^{n} \sum_{k=1}^{n} d_{j,k} x_j x_k $ denotes the quadratic form of the general programming probem.


## Support vector machines

Now that we have some understanding on QP and the general model we introduce the SVM model in its general form, first of all consider a data set of $N$ pairs in the form $(\text{x}_i, y_i)$, where $\text{x}_i \in \mathbb{R}^{p}$ and $y_i \in \{-1,1\}$. Without going deep into the assumptions and the idea of the model, the general optimization problem that solves the SVM model is the following one,

\begin{equation}
\min\limits_{\beta, \beta_0} \frac{1}{2} || \beta || ^2 + C \sum_{i=1}^{N} \epsilon_i
\end{equation}

\begin{equation}
\text{subject to:} \epsilon_i \geq 0, \ \ y_i (\text{x}_i^T \beta + \beta_0) \geq 1 - \epsilon_i, \ \ \forall i \in \{ 1,...,N \}
\end{equation}

where $C$ corresponds to a cost parameter that determines a trade-off between increasing the margin of the hyperplane generated in the model and labeling correctly each $y_i$ in the training stage. Moreover $||\beta||$ corresponds to the euclidean norm with the following form,

\begin{equation}
|| \beta ||^2 = (\sqrt{\beta_i^2 + ... + \beta_p^2})^2
\end{equation}

With this we can build our model and solve the model through a solver, for the purpose of this project we will solve it by using python and Gurobi, but you can try with other kind of approaches and solvers.

The first thing we will do is import the library with the solver for gurobipy and other packages useful for working with the problem

In [68]:
from sklearn import datasets # Allow for simulate data sets
import numpy as np # Matrix manipulation
from sklearn.model_selection import train_test_split # Separar en train y test
import pandas as pd
from gurobipy import *

In [69]:
# Simulation of a dataframe
X, y = datasets.make_blobs(

        n_samples = 100, # Number of samples
        n_features = 3, # Features
        centers = 2,
        cluster_std = 1
    )

y = np.where(y == 0, -1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Next step is creating the model

In [70]:
quadratic_model = Model('quadratic')

After creating the model we need to define the variables to be optimized in the model, these variables are,

$$ u = \beta $$
$$ c = \epsilon_i $$
$$ bi = \beta_0$$

In [71]:
u = {}
c = {}

for i in range(X_train.shape[1]):
    u[i] = quadratic_model.addVar(vtype = GRB.CONTINUOUS, name = 'u' + str(i))

for i in range(len(X_train)):
    c[i] = quadratic_model.addVar(vtype = GRB.CONTINUOUS, name = 'c' + str(i))

bi = quadratic_model.addVar(vtype = GRB.CONTINUOUS, name = 'bi')

quadratic_model.update()

Now we are gonna create the objective function and add it to the model

In [72]:
obj_fn = 0.5 * (sum(u[i]**2 for i in range(X_train.shape[1]))) + 1.0 * (sum(c[i] for i in range(len(X_train))))

quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)
quadratic_model.update()

Next step is to add constraints of the model

In [73]:
for i in range(len(X_train)):
    quadratic_model.addConstr(y_train[i] * (sum(u[j] * X_train[i][j] for j in range(X_train.shape[1])) - bi) - 1 + c[i] >= 0)
    quadratic_model.addConstr(c[i] >= 0)

Model has the objective function and constraints defined, now we can solve the model

In [74]:
quadratic_model.setParam('OutputFlag', True)
quadratic_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 140 rows, 74 columns and 420 nonzeros
Model fingerprint: 0x39c234f7
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-03, 1e+01]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 70 rows and 0 columns
Presolve time: 0.01s
Presolved: 70 rows, 74 columns, 350 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.415e+03
 Factor NZ  : 2.485e+03
 Factor Ops : 1.168e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Comp

We create a function that search for the variables we need related to $\beta_o$ and $\beta$

In [75]:
def find_letter(letter, lst):
    return any(letter in word for word in lst)

In [76]:
for v in quadratic_model.getVars():
    if find_letter('c', v.varName) == False:
        print('%s: %g' % (v.varName, v.x))

u0: 2.97994e-11
u1: 0.111919
u2: 0.141788
bi: 0.120362


In [77]:
predict_gurobi = []

for i in range(len(X_test)):
    predict_gurobi.append(sum(u[j].x*X_test[i][j] for j in range(X_train.shape[1])) - bi.x)

predict_gurobi = np.sign(predict_gurobi)

# Data frame con los tres resultados
resultados = pd.DataFrame({'Actual': y_test, 'Gurobi': predict_gurobi})
resultados['Gurobi'] = (resultados['Gurobi'].astype(int))
resultados

,Actual,Gurobi
0,-1,-1
1,-1,-1
2,-1,-1
3,-1,-1
4,1,1
5,-1,-1
6,1,1
7,1,1
8,1,1
9,1,1


### Conclusions

We have seen through this project that the SVM model corresponds to a quadratic programming problem that can be solved with any kind of nonlinear programming solvers such as GUROBI, GAMS, AMPL, and others. There is worth noting that the usage of such solvers is limited and this example is one with a small sample and few features, and extensions to the sample size and features is constrained to the licence of such programs and solvers, in case there is no possibility of using these solvers other approaches are applying heuristics that retrieve local optimal solutions to the problem and can be used as an approach for getting results